In [2]:
import json
import numpy as np
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from matplotlib.colors import ListedColormap
from matplotlib import rc
import random

ModuleNotFoundError: No module named 'pandas'

In [3]:
# matplotlib 한글 폰트 깨짐 문제 해결
import matplotlib.font_manager as fm
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
fontprop = fm.FontProperties(fname=font_path)

In [4]:
# 경로 설정

data_path = '/workspace/Data_All/'

img_path = data_path + 'image/'                     # /workspace/Data_All/image/
json_path = data_path + 'label/'                    # /workspace/Data_All/label/
segment_path = data_path + 'segmentation/'          # /workspace/Data_All/segmentation/

anger_img_path = img_path + 'anger/'                 # /workspace/Data_ALL/image/anger/
happy_img_path = img_path + 'happy/'                 # /workspace/Data_ALL/image/happy/
panic_img_path = img_path + 'panic/'                 # /workspace/Data_ALL/image/panic/
sadness_img_path = img_path + 'sadness/'             # /workspace/Data_ALL/image/sadness/

In [5]:
# 라벨 정보 json 파일 (train, val, test 파일 통합)
# train_sadness.json + val_sadness.json + test_sadness.json -> json_sadness

# json 파일 각각 불러오기
with open (json_path + "train_sadness.json", "r", encoding='euc-kr') as f:
    json_sadness_train = json.load(f)
with open (json_path + "val_sadness.json", "r", encoding='euc-kr') as f:
    json_sadness_val = json.load(f)
with open (json_path + "test_sadness.json", "r", encoding='euc-kr') as f:
    json_sadness_test = json.load(f)

# 하나의 파일로 통합
json_sadness = []
if isinstance(json_sadness_train, list):
    json_sadness.extend(json_sadness_train)
if isinstance(json_sadness_val, list):
    json_sadness.extend(json_sadness_val)
if isinstance(json_sadness_test, list):
    json_sadness.extend(json_sadness_test)

In [6]:
# Segmentation 정보 npz 파일 (train, val, test 통합된 파일)
# npz_sadness

npz_sadness = np.load(segment_path + 'combined_sadness.npz')

In [7]:
# json 파일 내 이미지 이름들과 npz 파일 내 이미지 이름들이 동일한지 확인

json_filenames = [item['filename'] for item in json_sadness]
npz_filenames = list(npz_sadness.keys())

# 동일 여부 확인
common_filenames = set(json_filenames) & set(npz_filenames)  # 교집합
only_in_json = set(json_filenames) - set(npz_filenames)      # JSON에만 있는 파일
only_in_npz = set(npz_filenames) - set(json_filenames)       # NPZ에만 있는 파일

# 결과 출력
print(f"공통 파일 개수: {len(common_filenames)}")
print(f"JSON에만 있는 파일 개수: {len(only_in_json)}")
print(f"NPZ에만 있는 파일 개수: {len(only_in_npz)}")

공통 파일 개수: 2100
JSON에만 있는 파일 개수: 0
NPZ에만 있는 파일 개수: 0


In [8]:
# npz 파일 확인

# 파일에 저장된 데이터 키 확인
keys = list(npz_sadness.keys())[:10]  # 처음 10개의 키만 가져옴
print("npz 파일 길이 : ", len(npz_sadness))
print("Keys in .npz file:", npz_sadness.keys())

# 각 키에 대한 배열 정보 출력
for key in keys:
    array = npz_sadness[key]
    print(f"\nKey: {key}")
    print(f"Shape: {array.shape}")
    print(f"Data type: {array.dtype}")
    print(f"Array contents:\n{array}")

npz 파일 길이 :  2100
Keys in .npz file: KeysView(NpzFile '/workspace/Data_All/segmentation/combined_sadness.npz' with keys: 00skd6f03c1bc8e3f618fb80b7863679bf684363a4d013ad634fab7c647ff8kws.jpg, 00yucd996df0c12e2dcac91e0a41d77a35181e5ff4cce66ea6cf61f05ec52d9hb.jpg, 01im33663229257659a232aa1a0a15a4c11740181881075072377a2a1fffbdeey.jpg, 028v4f9c9714b05ddb190ca7883cbd3e83b87fc4e7b341ba97bf08a75d0a04awj.jpg, 037n0dd389868164f7cf2736331868484540b863fc3fd474d44656a830279ug2l.jpg...)

Key: 00skd6f03c1bc8e3f618fb80b7863679bf684363a4d013ad634fab7c647ff8kws.jpg
Shape: (1932, 2576)
Data type: uint8
Array contents:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]]

Key: 00yucd996df0c12e2dcac91e0a41d77a35181e5ff4cce66ea6cf61f05ec52d9hb.jpg
Shape: (2320, 3088)
Data type: uint8
Array contents:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]]

Key: 01im3366322925765

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

filenames = npz_filenames  # npz_filenames = list(npz_sadness.keys())

for filename in filenames[:30]:
    data = npz_sadness[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(sadness_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (모든 부분을 검정색으로 초기화)
    new_img = image.copy()  # 원본 이미지 복사

    # 모든 영역을 검정색으로 초기화
    if len(image.shape) == 2:  # 단일 채널 이미지
        new_img[:, :] = 0
    else:  # RGB 이미지
        new_img[:, :] = [0, 0, 0]

    # Segmentation 데이터에서 얼굴 영역(값이 3)을 추출
    mask = data == 3
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와서 새로운 이미지에 덮어씌움

    # 시각화
    plt.figure(figsize=(12, 6))

    # 원본 이미지
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지 시각화
    plt.subplot(1, 2, 2)
    plt.imshow(new_img)
    plt.title("Face Only (Rest Black)")
    plt.axis("off")

    plt.suptitle(f"File : {filename}")
    plt.show()



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# 경로 설정
data_path = '/workspace/Data_All/'
img_path = data_path + 'image/'                     # /workspace/Data_All/image/
segment_path = data_path + 'segmentation/'          # /workspace/Data_All/segmentation/

sadness_img_path = img_path + 'sadness/'  # 원본 이미지 경로
output_path = '/workspace/Data_Augumentation/segmentation_face/image/train/sadness/'  # 결과 저장 경로

# npz 파일 로드 (파일 경로에 맞게 수정)
npz_sadness = np.load(segment_path + 'combined_sadness.npz')  # npz 파일 경로 수정
npz_filenames = list(npz_sadness.keys())  # npz 파일에서 모든 파일 이름을 리스트로 가져옴

# 첫 30개 파일을 처리
for filename in npz_filenames[:30]:
    data = npz_sadness[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(sadness_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (모든 부분을 검정색으로 초기화)
    new_img = image.copy()  # 원본 이미지 복사

    # 모든 영역을 검정색으로 초기화
    if len(image.shape) == 2:  # 단일 채널 이미지
        new_img[:, :] = 0
    else:  # RGB 이미지
        new_img[:, :] = [0, 0, 0]

    # Segmentation 데이터에서 얼굴 영역(값이 3)을 추출
    mask = data == 3  # 얼굴이 3으로 지정되어 있다고 가정
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와서 새로운 이미지에 덮어씌움

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지를 640x640으로 리사이징 후 저장
    output_file_path = os.path.join(output_path, f"face_only_{filename}")  # 저장할 경로 설정
    new_img_bgr = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷을 사용하므로 변환

    # 640x640으로 리사이즈
    resized_img = cv2.resize(new_img_bgr, (640, 640))

    # 리사이즈된 이미지 저장
    cv2.imwrite(output_file_path, resized_img)  # 이미지 저장

    # 시각화
    plt.figure(figsize=(12, 6))

    # 원본 이미지
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지 시각화
    plt.subplot(1, 2, 2)
    plt.imshow(new_img)
    plt.title("Face Only (Rest Black)")
    plt.axis("off")

    plt.suptitle(f"File : {filename}")
    plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

# 경로 설정
data_path = '/workspace/Data_All/'
img_path = data_path + 'image/'  # /workspace/Data_All/image/
segment_path = data_path + 'segmentation/'  # /workspace/Data_All/segmentation/

sadness_img_path = img_path + 'sadness/'  # 원본 이미지 경로
output_path = '/workspace/Data_Augumentation/segmentation_face/image/train/sadness/'  # 결과 저장 경로
output_label_path = '/workspace/Data_Augumentation/segmentation_face/label/train/sadness/'  # 레이블 저장 경로

# 저장 경로 생성
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_label_path, exist_ok=True)

# npz 파일 로드
npz_sadness = np.load(segment_path + 'combined_sadness.npz')  # npz 파일 경로 수정
npz_filenames = list(npz_sadness.keys())  # npz 파일에서 모든 파일 이름을 리스트로 가져옴

# JSON 파일 로드 (sadness 관련 JSON 파일 로드)
json_path = data_path + 'label/'  # JSON 경로 설정
with open(json_path + "train_sadness.json", "r", encoding='euc-kr') as f:
    json_sadness_train = json.load(f)

# JSON 파일에서 레이블 데이터 매핑
label_data = {item['filename']: item for item in json_sadness_train}

# JSON 파일 하나에 모든 레이블 데이터를 저장
all_labels = []

# 이미지 및 레이블 처리
for filename in npz_filenames[:30]:  # 첫 30개 파일 처리
    data = npz_sadness[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(sadness_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (검정색 초기화)
    new_img = np.zeros_like(image)  # 검정색으로 초기화
    mask = data == 3  # 얼굴 영역 추출
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와 새로운 이미지에 덮어씌움

    # 마스크 리사이즈
    mask_resized = cv2.resize(mask.astype(np.uint8), (640, 640), interpolation=cv2.INTER_NEAREST)  # 마스크 리사이즈

    # 리사이즈된 이미지 생성
    resized_img = np.zeros((640, 640, 3), dtype=np.uint8)
    resized_img[mask_resized == 1] = cv2.resize(new_img, (640, 640))[mask_resized == 1]  # 리사이즈된 마스크에 맞춰 얼굴 영역만 배치

    # 리사이즈된 이미지 저장
    resized_img_bgr = cv2.cvtColor(resized_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷 사용
    output_file_path = os.path.join(output_path, f"face_only_{filename}")
    cv2.imwrite(output_file_path, resized_img_bgr)  # 리사이즈된 이미지 저장

    # 레이블 데이터 업데이트
    if filename in label_data:
        label_info = label_data[filename]
        
        # 원본 이미지의 크기
        original_height, original_width = image.shape[:2]
        
        # 리사이즈된 이미지의 크기
        new_width, new_height = 640, 640
        
        # 바운딩 박스 비례적으로 크기 조정
        for annot in ['annot_A', 'annot_B', 'annot_C']:
            if annot in label_info:
                box = label_info[annot]['boxes']
                box['minX'] = int(box['minX'] * (new_width / original_width))
                box['minY'] = int(box['minY'] * (new_height / original_height))
                box['maxX'] = int(box['maxX'] * (new_width / original_width))
                box['maxY'] = int(box['maxY'] * (new_height / original_height))
        
        label_info['resized'] = True  # 리사이즈 여부 추가

        # 레이블 데이터를 JSON 리스트에 추가
        all_labels.append(label_info)

    # 시각화 (선택)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(resized_img)
    plt.title("Face Only (Rest Black) Resized")
    plt.axis("off")
    plt.suptitle(f"File: {filename}")
    plt.show()

# 모든 레이블을 하나의 JSON 파일로 저장
output_label_file = os.path.join(output_label_path, "combined_labels.json")
with open(output_label_file, "w", encoding="utf-8") as json_file:
    json.dump(all_labels, json_file, ensure_ascii=False, indent=4)

print(f"Labels saved to: {output_label_file}")

     
